In [ ]:
#-- Install ultralytics for YOLO  --------------------------------------------------------------------------
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Import -----------------------------------------------------------------------------------------------
from ultralytics import YOLO

import cv2
import matplotlib.pyplot as plt

import os
import shutil
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Initialize ---------------------------------------------------------------------------------------------------
out_path = '/kaggle/working/'

videos_dir = '/kaggle/input/moving-objs-1/'

results_dir = '/kaggle/working/result_videos/'

MOTION_THRESHOLD = 10
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Create Dir for saving Results ---------------------------------------------------------------------------------
os.makedirs(results_dir, exist_ok=True)
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Initialize YOLOv8 and background subtractor ------------------------------------------------------------------
model = YOLO('yolov8n.pt')
back_sub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=False)
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
for video_file in os.listdir(videos_dir):   
    
    
    
    #-- log --
    print(f'Processing {video_file} ==========================================================')
    
    #-- Create Folder for saving results --
    dot_index = video_file.rfind('.')   
    video_result_dir_name = 'result_for_' + video_file[:dot_index]
    video_result_dir_path = results_dir + video_result_dir_name + '/'
    os.makedirs(video_result_dir_path, exist_ok=True)
        
    #-- load video --
    video_path = os.path.join(videos_dir, video_file)    
    video = cv2.VideoCapture(video_path)
    
    #-- Get number of frames -- 
    number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    #-- Get the width and height of the frames --
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv2.CAP_PROP_FPS))

    #-- Initialize VideoWriter to save the output video --
    result_video = cv2.VideoWriter(video_result_dir_path + video_file[:dot_index] + '.avi',
                                   cv2.VideoWriter_fourcc(*'XVID'),
                                   fps,
                                   (frame_width, frame_height))    
    
    
    #-- Run YOLO Frame by Frame --
    frame_number = 0
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        
        frame_number += 1
        
        #-- Apply background subtraction --
        fg_mask = back_sub.apply(frame)
        
        
        #-- show some frames --
        if frame_number % (number_of_frames//5) == 0:
            plt.figure(figsize=(5, 5))
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.title(f'main frame - frame number={frame_number}')
            file_name = f'main_frame_{frame_number}.png'
            plt.savefig(video_result_dir_path + file_name)
#             plt.show()

            plt.figure(figsize=(5, 5))
            plt.imshow(cv2.cvtColor(fg_mask, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.title(f'fg_mask - frame number={frame_number}')
            file_name = f'fg_mask_{frame_number}.png'
            plt.savefig(video_result_dir_path + file_name)
#             plt.show()
        
        #-- Detect objects with YOLOv8 --
        results = model(frame)
        
        #-- split only moving objects --
        moving_objects = []
        for result in results:
            for box in result.boxes:  
                class_id = int(box.cls) 
                bbox = box.xyxy.tolist()[0]            
                x1, y1, x2, y2 = map(int, bbox)

                #-- Check if the detected object has motion --
                if fg_mask[y1:y2, x1:x2].mean() > MOTION_THRESHOLD:  
                    moving_objects.append((x1, y1, x2, y2, class_id))  


        #-- plot bounding box for moving objects on the frame --
        for (x1, y1, x2, y2, class_id) in moving_objects:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  #-- Green box with thickness 2 --
            
            label = f"Class {class_id}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        result_video.write(frame)

        #-- show some frames --
        if frame_number % (number_of_frames//5) == 0:
            plt.figure(figsize=(10, 10))
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.title(f'moving objcs - frame number={frame_number}')
            file_name = f'moving_objcs_{frame_number}.png'
            plt.savefig(video_result_dir_path + file_name)
            #plt.show()   
    
        
        
    shutil.make_archive(out_path+video_result_dir_name, 'zip', video_result_dir_path)
    print(f'number_of_frames: {number_of_frames}' ) 

    video.release()
    result_video.release()  


shutil.rmtree(results_dir)
display.clear_output()   
print(':)')
#-----------------------------------------------------------------------------------------------------------------    